### 진행 과정
1. 파일 로드 및 결측치 제거
2. 전처리 함수: 한자 제거
3. 2번의 과정으로 생긴 결측치 제거
4. 의미없는 문장 제거 함수 / 인덱스 정렬
5. Description 컬럼의 의미없는 문장 인덱스 제거(수작업)
6. 파일 저장

### 1. 파일 로드 및 결측치 제거

In [ ]:
import pandas as pd
import numpy as np
import glob
import re

In [ ]:
file_paths = ["museum_data_1.csv", "museum_data_2.csv", "museum_data_3.csv", "museum_data_4.csv", "museum_data_5.csv", "museum_data_324.csv", "museum_data_35640.csv"]

df_list = [pd.read_csv(file) for file in file_paths]
df_merged = pd.concat(df_list, ignore_index=True)
print(len(df_merged))

df_merged = df_merged.drop_duplicates(subset=["Title"])
print(len(df_merged))

df_merged = df_merged.dropna(subset=["Title", "Description"]) 
print(len(df_merged))

print(f"최종 데이터 개수: {len(df_merged)}")

df_merged.to_csv("merged_museum_data.csv", index=False)

### 2. 전처리 함수: 한자 제거

In [ ]:
def keep_korean_only(text):
    if isinstance(text, str):
        text = re.sub(r'[^가-힣0-9\s]', '', text) 
        text = re.sub(r'\s+', ' ', text).strip()
    return text

df_merged["Title"] = df_merged["Title"].apply(keep_korean_only)
df_merged["Description"] = df_merged["Description"].apply(keep_korean_only)

df_merged.info()

cleaned_file_path = "merged_museum_data.csv"
df_merged.to_csv(cleaned_file_path, index=False)


print("✅ 한국어만 남기고 전처리 완료!")



<class 'pandas.core.frame.DataFrame'>
Index: 7751 entries, 6 to 36011
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        7751 non-null   object
 1   Image URL    7751 non-null   object
 2   Description  7751 non-null   object
dtypes: object(3)
memory usage: 242.2+ KB
✅ 한국어만 남기고 전처리 완료!


### 3. 2번 과정에서 생긴 결측치 제거

In [ ]:
file_path = "merged_museum_data.csv"
df_merged = pd.read_csv(file_path) 

df_merged[["Title", "Description"]] = df_merged[["Title", "Description"]].replace(r'^\s*$', np.nan, regex=True)  
df_merged = df_merged.dropna(subset=["Title", "Description"])
df_merged.info()

cleaned_file_path = "merged_museum_data.csv"
df_merged.to_csv(cleaned_file_path, index=False)

print(f"🔹 파일이 저장되었습니다: {cleaned_file_path}")

<class 'pandas.core.frame.DataFrame'>
Index: 7619 entries, 0 to 7750
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Title        7619 non-null   object
 1   Image URL    7619 non-null   object
 2   Description  7619 non-null   object
dtypes: object(3)
memory usage: 238.1+ KB
🔹 파일이 저장되었습니다: merged_museum_data.csv


### 4. 의미없는 문장 제거 함수 / 인덱스 정렬

In [ ]:
def remove_incomplete_sentences(text):
    if isinstance(text, str):
        meaningless_patterns = r'^\s*(달고|달았음|으로|로서|이고|한|있음|만|마다|새긴|달었음|록로|팠음|폭|은|에는||을|를|에|과|와|및|의|로|에서|과 함께|의 경우|을 위해|을 통해|및 에선|을 하였음|에 있음|을 들었음|으로 하였음|및 으로|을 하여|있음|달았음|하였음)\s*$'
        if re.fullmatch(meaningless_patterns, text):
            return np.nan 
    return text 

df_merged["Description"] = df_merged["Description"].apply(remove_incomplete_sentences)
print(len(df_merged))

df_merged = df_merged.dropna(subset=["Description"])
df_merged = df_merged.reset_index(drop=True)
print(len(df_merged))

cleaned_file_path = "merged_museum_data.csv"
df_merged.to_csv(cleaned_file_path, index=False)

print(f"🔹 파일이 저장되었습니다: {cleaned_file_path}")

7619
7615
🔹 파일이 저장되었습니다: merged_museum_data.csv


### 5. Description 컬럼의 의미없는 문장 인덱스 제거(수작업)

In [ ]:
df_short_desc = df_merged[df_merged["Description"].str.len() <= 20]

print(f"🔹 총 {len(df_short_desc)}개의 행이 필터링되었습니다.")
print(df_short_desc['Description'].head(20))


🔹 총 147개의 행이 필터링되었습니다.
25       용의 정수리에 최자가 새겨져 있다
26                 왕 명문이 있다
49              손잡이에 초문이 있다
62           원판번호 66319 청동추
69      측면에는 장사군전 글자가 양각되었다
87        원판번호 61914 나전길상문함
233           원판번호 70610 장도
270          원판번호 290032 한경
273    원판번호 97113 격납상자제목 종루
329            원판번호 7656 문서
347       앞면은 상평오 글자가 찍혀 있다
368    원판번호 300113 도서자료 지석묘
372                새겨진 문양 등
382        원판번호 7209 통도사탑비명
398      원판번호 8564 격납상자제목 병
467            바닥에 정이 음각되었다
491       내면 가운데 장흥고를 인화하였다
505            원판번호 6475 서간
534     외면에 려 모 을 등의 문자가 있다
576         원판번호 6207 귀형화약통
Name: Description, dtype: object


### 6. 5번에서 거른 문장 삭제 후 파일 저장

In [ ]:
df_short_desc = df_short_desc.drop(index=[29, 83, 157, 180, 190, 192, 238, 256, 275, 280, 281, 286, 318, 331, 360, 395, 396, 397, 408, 429])
df_short_desc = df_merged.reset_index(drop=True) 
cleaned_file_path = "museum_data_6_final.csv"
df_short_desc.to_csv(cleaned_file_path, index=False)

print(f"🔹 파일이 저장되었습니다: {cleaned_file_path}")


🔹 파일이 저장되었습니다: museum_data_6_final.csv
